In [329]:
import os
import re
from datetime import datetime
import pandas as pd

In [330]:
def oscrapuj_le(slozka, soubor):

    def mista(i):
        return int(spojeni.split(">")[index + 2].split('<')[0].strip().replace('&gt; 20','21'))
    
    with open(os.path.join(slozka,soubor), "r", encoding="utf-8") as spojeni:
        spojeni = spojeni.read()
    spoje = []
    spoj = None
    pocitadlo_casu = 1
    oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
    oscrapovano = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
    for index, radek in enumerate(spojeni.split(">")):
        format_data = r"\w{4,10}, \d{2}\. \d{2}\. 202\d"
        if re.search(format_data, radek):
            datum = re.search(format_data, radek).group(0)
            den = datum.split(',')[0].strip().lower()
            datum = datum.split(',')[1].strip().replace(' ','')
        if re.search(r"\d{2}:\d{2}", radek[0:5]):
            pocitadlo_casu += 1
            if pocitadlo_casu % 2 == 0:
                if spoj:
                    spoj['oscrapovano'] = oscrapovano
                    spoj['predstih'] = spoj['odjezd'] - oscrapovano
                    spoj['prodejce'] = 'LE'
                    spoj['vlaky'] = set(spoj['vlaky'])
                    try:
                        spoj['volnych_mist'] = spoj['volna_mista_economy'] + spoj['volna_mista_economy_plus'] + spoj['volna_mista_economy_business'] + spoj['volna_mista_premium']
                    except:
                        pass
                    spoje.append(spoj)
                spoj = {}
                spoj['vlaky'] = []
                spoj['odkud'] = None
                cas = radek[0:5]
                spoj['odjezd'] = datetime.strptime('.'.join(datum.split('.')) + " " + cas.strip(), "%d.%m.%Y %H:%M")
        if ("Ostrava" in radek) or ("Praha" in radek[0:10]) or ("Pardubi" in radek[0:10]) or ("Kraków" in radek[0:10]) or ("Košice" in radek[0:10]):
            if spoj != None:
                if spoj['odkud'] == None:
                    spoj['odkud'] = radek.split("<")[0].strip()
                else:
                    spoj['kam'] = radek.split("<")[0].strip()            
        if "&nbsp;Kč" in radek:
            spoj['cena'] = re.search(r'\d{1,5}', radek.replace("&nbsp;","").split('|')[-1]).group()
        if " km<" in radek:
            spoj['vzdalenost'] = int(re.search(r'\d{1,5}', radek).group(0))
        if re.search(r'\d{1,2} h \d{1,2} min', radek):
            jizdni_doba = radek.split('|')[0]
            jizdni_doba = jizdni_doba.split('h')
            h = re.search(r"\d{1,2}", jizdni_doba[0]).group()
            min = re.search(r"\d{1,2}", jizdni_doba[1]).group()
            spoj['jizdni_doba'] = (int(h) * 60) + int(min)
        if "Economy<" in radek[0:10]:
            spoj['volna_mista_economy'] = mista(index) # spojeni.split(">")[index + 2].split('<')[0].strip()
        if "Economy P" in radek[0:10]:
            spoj['volna_mista_economy_plus'] = mista(index) # spojeni.split(">")[index + 2].split('<')[0].strip()
        if "Business" in radek[0:10]:
            spoj['volna_mista_economy_business'] = mista(index) # spojeni.split(">")[index + 2].split('<')[0].strip()
        if "Premium<" in radek[0:10]:
            spoj['volna_mista_premium'] = mista(index) # spojeni.split(">")[index + 2].split('<')[0].strip()
        if re.search(r"LE\d{2}", radek[0:4]):
            spoj['vlaky'].append(re.search(r"LE\d{1,7}", radek).group())
    return spoje

In [331]:
oscrapuj_le("downloads/2024-11-08","le_Praha_Krak%C3%B3w%20G%C5%82%C3%B3wny_D04_2024-11-08_20-16-21.html")

[{'vlaky': set(),
  'odkud': 'Praha hl.n.',
  'odjezd': datetime.datetime(2024, 11, 15, 16, 0),
  'kam': 'Kraków Główny',
  'cena': '1279',
  'jizdni_doba': 361,
  'oscrapovano': datetime.datetime(2024, 11, 8, 20, 16, 21),
  'predstih': datetime.timedelta(days=6, seconds=71019),
  'prodejce': 'LE'}]

In [337]:
kam = "data" 
os.makedirs(kam, exist_ok=True)
hotove = [y for y in os.listdir(kam) if y[0:3] == "le_"] 
for x in os.listdir("downloads"):
    nazev_souboru = "le_" + x + ".parquet"
    if nazev_souboru not in hotove:
        den = []
        le = [y for y in os.listdir(f"downloads/{x}") if y[0:3] == "le_"] 
        print(f"{x}: {len(le)}")
        for y in le:
            print(y)
            den = den + oscrapuj_le(f"downloads/{x}",y)
        if len(den) > 0:
            df_den = pd.DataFrame(den).sort_values(by="oscrapovano").reset_index(drop=True)
            df_den.to_parquet(os.path.join(kam,nazev_souboru))

2024-10-31: 0
2024-11-01: 0
2024-11-02: 0
2024-11-03: 0
2024-11-04: 0
2024-11-05: 0
2024-11-06: 0
2024-11-09: 287
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_06-25-15.html
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_08-26-05.html
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_10-27-06.html
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_12-27-46.html
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_16-19-41.html
le_Ko%C5%A1ice_Pardubice_D00_2024-11-09_18-22-49.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_06-25-41.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_08-26-33.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_10-27-34.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_12-28-11.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_16-20-08.html
le_Ko%C5%A1ice_Pardubice_D01_2024-11-09_18-23-17.html
le_Ko%C5%A1ice_Pardubice_D02_2024-11-09_06-26-06.html
le_Ko%C5%A1ice_Pardubice_D02_2024-11-09_08-26-58.html
le_Ko%C5%A1ice_Pardubice_D02_2024-11-09_10-27-58.html
le_Ko%C5%A1ice_Pardubice_D02_2024-11-09_12-28-36.html
le_Ko%C5%A1ice_Pardubi